This Markdown objective is to find the rent from the df column named infos. this column has many problems namely the enter is not the same. 

In [123]:
import pandas as pd
import nltk
nltk.download('punkt')
import numpy as np


[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [124]:
df = pd.read_csv("https://data.enseignementsup-recherche.gouv.fr/explore/dataset/fr_crous_logement_france_entiere/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false", sep = ';')

Part I : Initialisation pour détecter la présence de symbole dans les mots. 

On va créer une fonction qui prend en argument un mot et une liste qui va déctecter s il y a un des élèments de cette liste dans le mot. Cette fonction va ensuite dans notre fonction principale. 

In [129]:
List_check = ["euros","€", "Loyer", "Loyers", "loyer", "loyers", "tarifs", "Tarifs", "tarif","Tarif"]

In [5]:
"""
String_checking est une fonction qui prend un mot en argument et un élèment et retourne 0 si l'élèmenet n'est 
pas inclus dans le mot et 1 inversement. 
"""


def string_checking(string, check):
    result = 0 
    if string.find(check) == -1:
        result = 0
    else: 
        result = 1
    return result


In [7]:
"""
Cette fonction prend en argument un mot à tester et une liste. Elle va utiliser la fonction précédente pour
vérifier qu'il n'y a aucun élèment de la liste dans notre mot. De même elle retourne 0 ou 1 en fonction de
s'il y a un élèment ou non. 
"""

def word_ban(string, liste):
    result = 0 
    for check in liste: 
        if string_checking(string, check) == 1: 
            result = 1 
        else: 
            result = 0
    return result 
            

Part II : Main functions 

README 

On va créer une fonction pour trouver le loyer à partir de la fonction précèdente. La fonction prend pour argument un texte (ici df['infos'][j] qui correspond au texte informatif d'une résidence où se trouve le loyer. La fonction va nettoyer le texte d'une manière très précise : 

1 - On va enlever tout les espaces du texte de façon à n'avoir qu'un block uniforme de texte. 
2 - on va enlever les parenthèses (problèmes souvent eu - à voir si c'est toujours le cas sans) 
3 - On va enlever les petits 2 des mètres carrés qui posent souvent dans la suite du code. 
4 - On va remplacer les à en € de façon à avoir les loyers qui sont inscrits de la manière suivante : 200 à 350€ (notrer recherche se fonde grâce au €). 
5 - On va remplacer les 'euros' en €
6 - on va remplacer les €€ en € car il peut y avoir des '200€à350€' qui deviennent '200€€350€'et nous on veut '200€350€'. 
7 - On va rajouter un espace après chaque '€'. 
8 - On normalise les , en . pour avoir les mêmes typologies
9 - on enlever les : (à voir si sa fonctionne toujours sans). 

On va ensuite tokeniser notre block de texte grâce à la fonction word_tokenize du package nltk. On va ainsi passer de blocks de caractères séparé par un espace à une liste de mots. 

Exemple (df["infos"][0]): 

On passe de :

"Logements HLM

70 T1 : 325 euros
14 T1bis de 30m² : 373,50 euros
13 T2 de 37m² : 404 euros

   
   
   
 Photos de la résidence"
   
à : 

"['LogementsHLM', '70T1', '325€', '14T1bisde30m373.50€', '13T2de37m404€', 'Photosdelarésidence', 'https//www.flickr.com/photos/crousaixmarseilleavignon/albums/72157679829208245']"

L'idée va ensuite de trouver le loyer grâce à cette liste de mot. On va ainsi appliquer la fonction rent. cette fonction prend en argument cette liste de mots. Pour chaque élèments de notre liste on va d'abord chercher l'index de € (s'il existe et s'il n'appartient pas à la liste des mots interdits - on ne veut pas les moments des cautions par exemple). On va ensuite regarder si le caractère à la gauche du € est un chiffre ou non. Si c'est un chiffre on va réetirer cela pour le caractère à gauche de ce dernier. Ainsi de suite jusqu'à trouver un élèment qui n'est pas un chiffre. Dans ce cas là on va arrêter et prendre le nombre entre les deux indices. Le problème est celui des virgule : on rajoute une condition qui dit que si on trouve une virgule on regarde à gauche si c'est un chiffre. On obtient ainsi une liste des loyers avec quelques valeurs abberrantes que l'on gera après. 



In [152]:
def rent(word):
    Loyer = [] #On crée une liste qui stockera le loyer
    #try:
    for i in range(0,len(word)): #on regarde dans chaque block de caractère 
        index = word[i].find("€") #on cherche l'index de l'euros normalement pour tout les loyers il y est 
        search = 0 #il nous servira à arrêter la boucle while 
        if word_ban(word[i],List_banned_word) == 0 and word[i].find("€") != -1: #On regarde si dans le block de mot il n'y a pas un mot banni comme APL pour éviter d'avoir le montant de ceux-ci, ce qui fausserait nos résultats.
            while search == 0: #Tant que notre variable de contrôle est égale à 0, on va chercher le loyer (ie : tant que ce sont des chiffres, on va chercher la nature (chffre ou non) de ce qui se situe à gauche.
                search_index = word[i][index - 1] #On cherche l'index de ce qui se trouve à gauche de l'euros
                try: #Try permet de tester si c'est un chiffre tout en ne bloquant pas les opérations avec une erreure
                    search_index = int(search_index) #On regare si à gauche de l'ancien index c'est un entier ou non (Dans le cas contraire on sera dans except).
                    index = index - 1 #on remonte de 1 l'index pour voir si ce qui se trouve à gauche de notre chiffre est également un chiffre.
                except: #si ce n'est pas un entier à gauche de l'euros ou du dernier chiffre identifié. 
                    if word[i][index - 1] == ".": #on regarde si ce n'est pas un point comme dans 350.85 euros. Si on ne met pas cette ligne on aurait seulement 85 et non 350.85. 
                        index = index - 2 #On va regarde ce qui est avant le point pour trouver le reste du montant
                        search_index = word[i][index - 1] #Sysiphe remonte sa pierre et recommence le même processus qu'avant
                        try: #on regarde de nouveau si à gauche du point c'est un entier
                            search_index = int(search_index) 
                            index = index - 1 
                        except:
                            search = 1 #si ce n'est pas un chiffre ie une lettre on arrête
                            index = -1 #permet de ne pas avoir des erreurs du style .50
    
                    else: #si ce n'est pas un point on arrête le compte. 
                        search = 1
                #print(word[i][index : word[i].find("euros")])

            if word[i][index : word[i].find("€")]!= '': #on regarde le fait que le token ne soit pas juste un '€'
                Loyer.append(float(word[i][index : word[i].find("€")])) #On ajoute au loyer le loyer trouvé
    #except:
        #if word[i] != "€" :
            #List_banned_word.append(word[i])
            #print(List_banned_word)
            #rent(word)
    return Loyer

In [165]:
def rent_find(residence): 
    residence = residence.replace(' ', '')
    residence = residence.replace('(', '')
    residence = residence.replace(')', '')
    residence = residence.replace('²', '')
    residence = residence.replace('m²à', '')
    residence = residence.replace('à', '€')
    residence = residence.replace('euros', '€')
    residence = residence.replace('€€', '€')
    residence = residence.replace('€', '€ ')
    residence = residence.replace(',', '.')
    residence = residence.replace(':', '')
    word =  nltk.word_tokenize(residence, language='french') #on transforme notre scripts en block de mots. ceux qui nous intéressent ont un euros à la fin (mais il y en a qui ont un euros à la fin et qui ne nous intéresse pas
    return rent(word)
    

Part III : construction du Dataframe 

on crée une boucle for sur toute les résidences. si la fonction foctionne où l'ajoute à une liste sinon on ajoute NAN et on crée la colone loyer. 

In [166]:
a = 0
b = 0
Loyer = []
for j in range(0,df.shape[0]):
    try:
        Loyer.append(rent_find(df["infos"][j]))
        b = b + 1
    except: 
        Loyer.append("NAN")
        a = a + 1
df['Loyer'] = Loyer
print("échec pour ",a)
print("réussite pour ", b)
print("total ", a + b)

échec pour  11
réussite pour  777
total  788


In [184]:
import math

a = df['Loyer'][7]


print(a)
print(len(a))


[472.19, 260.0, 350.0, 517.47, 370.0, 621.94, 450.0]
7


In [212]:
def remove_fun(residence):
    lenght = residence
    for i in range(0, len(lenght)):
        num = int(residence[i])
        num = str(num)
        if len(num) <= 2 or len(num) >= 4:
            residence.remove(residence[i])
    return residence

    
               

In [222]:
remove_fun(df['Loyer'][10])

[]

In [215]:
for j in range(0,df.shape[0]):
    if df["Loyer"][j] != "NAN":
        try : 
            remove_fun(df["Loyer"][j])
        except: 
            try: 
                remove_fun(df["Loyer"][j])
            except: 
                remove_fun(df["Loyer"][j])
                try: 
                    remove_fun(df["Loyer"][j])
                except: 
                    remove_fun(df["Loyer"][j])

In [223]:
Max = []
Min = []
Mean = []
for j in range(0,df.shape[0]):
    if df["Loyer"][j] != "NAN" or df["Loyer"][j] != []:
        print(j)
        Max.append(max(df["Loyer"][j]))
        Min.append(min(df["Loyer"][j]))
        Mean.append(np.mean(df["Loyer"][j]))
    else: 
         Max.append("NAN")
         Min.append("NAN")
         Mean.append("NAN")
df['Max Loyer'] = Max
df['Min Loyer'] = Min
df['Mean Loyer'] =  Mean

0
1
2
3
4
5
6
7
8
9


ValueError: max() arg is an empty sequence

In [216]:
df

,id,title,short_desc,zone,infos,services,contact,address,mail,phone,openinghours,interneturl,appointmenturl,virtualvisiturl,bookingurl,troubleshootingurl,geocalisation,photo,regions,Loyer
0,939,Résidence Canada,Proche du campus de la santé de la Timone,Marseille,Logements HLM\n\n70 T1 : 325 euros\n14 T1bis d...,"parking, interphone",Résidence Canada \r\n 31 rue du Canada - 13010...,31 rue du Canada - 13010 Marseille,NaN,04 91 47 48 76,NaN,https://www.crous-aix-marseille.fr/,https://www.crous-aix-marseille.fr/,https://www.flickr.com/photos/crousaixmarseill...,https://www.crous-aix-marseille.fr/logements/c...,https://www.crous-aix-marseille.fr/,"43.2862205505,5.4006781578",https://www.flickr.com/photos/crousaixmarseill...,Provence-Alpes-Côte d'Azur,"[325.0, 373.5, 404.0]"
1,940,Résidence Les Petites Maries,Centre ville. À proximité du lycée Thiers,Marseille,"Immeuble ancien, entièrement rénové, situé...",interphone,Résidence Les Petites Maries8 rue des Petites ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"43.3005104065,5.3773541451",NaN,Provence-Alpes-Côte d'Azur,"[244.54, 309.07, 487.35]"
2,946,Résidence Laugier,À proximité de l'université d'Avignon,Avignon,"75 T1 individuels de 19 à 23 m² à 323,60 € - D...","Équipements collectifs : laverie, contrôle d'...",Résidence Laugier \r\n 100 rue Carreterie – 84...,100 rue Carreterie – 84000 Avignon,antenne.avignon@crous-aix-marseille.fr,NaN,NaN,https://www.crous-aix-marseille.fr/,https://www.crous-aix-marseille.fr/,https://www.flickr.com/photos/crousaixmarseill...,https://www.crous-aix-marseille.fr/logements/c...,https://www.crous-aix-marseille.fr/,"43.9510726929,4.8152480125",https://www.flickr.com/photos/crousaixmarseill...,Provence-Alpes-Côte d'Azur,"[323.6, 180.0, 390.1, 240.0]"
3,949,Résidence Les 3 clés d Or,À proximité de l'université d'Avignon,Avignon,"66 T1 de 15,83 à 19,80 m² - 322,84 à 378,30€ C...","Équipements collectifs : laverie, garage 2 rou...",Résidence Les 3 Clés d'Or1E boulevard Paul Mar...,1E boulevard Paul Mariéton - 84000 Avignon,antenne.avignon@crous-aix-marseille.fr,NaN,NaN,https://www.crous-aix-marseille.fr/,https://www.crous-aix-marseille.fr/,https://www.flickr.com/photos/crousaixmarseill...,https://www.crous-aix-marseille.fr/logements/c...,https://www.crous-aix-marseille.fr/,"43.9515228271,4.8247108459",https://www.flickr.com/photos/crousaixmarseill...,Provence-Alpes-Côte d'Azur,"[322.84, 378.3, 220.0, 270.0, 371.96, 401.3, 2..."
4,953,Résidence Arc de Meyran - Coton Rouge,À proximité de l’IUT et de la fac de lettres,Aix-en-Provence,"22 studios de 20 m² : 226,70 euros - Dépôt de ...",Parking,Résidence Arc de Meyran \r\n 590 avenue du Cot...,590 avenue du Coton Rouge – 13621 Aix-en-Prove...,cu.arcdemeyran@crous-aix-marseille.fr,04 42 93 64 75,NaN,https://www.crous-aix-marseille.fr/,https://www.crous-aix-marseille.fr/,NaN,https://www.crous-aix-marseille.fr/logements/c...,https://www.crous-aix-marseille.fr/,"43.5132026672,5.4528851509",NaN,Provence-Alpes-Côte d'Azur,"[226.7, 180.0, 250.57, 180.0, 509.8, 400.0, 71..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,1042,Cité Campus 1 et 2,campus universitaire du Moufia à Sainte-Clotilde,Nord,Localisation : Campus universitaire du Mouf...,Descriptions : 200 Logements,Cité Campus 1 et 2 Campus universitaire du M...,avenue René Cassin 97490 Sainte-Clotilde,NaN,NaN,NaN,https://www.crous-reunion.fr/,https://www.crous-reunion.fr/,NaN,https://www.crous-reunion.fr/logements/constit...,https://www.crous-reunion.fr/,"-20.9012107849,55.485912323",https://admin-v2.crous-mobile.fr//media/crous-...,La Réunion,[]
784,934,Cité U Galinat,En face du campus de la santé de la Timone,Marseille,312 chambres de 9m² avec wc et douches en comm...,"Équipements collectifs : wc, douches, cuisine...",Cité U Galinat \r\n 38 boulevard Jean Moulin -...,38 boulevard Jean Moulin - 13392 Marseille ced...,cu.galinat@crous-aix-marseille.fr,04 91 29 62 50,NaN,https://www.crous-aix-marseille.fr/,https://www.c

In [15]:
df.shape

(788, 19)